# Рекомендательные системы

## Урок 4. Рекомендательные системы на основе контента

## Практическое задание: написать заготовки функций

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import bsr_matrix

In [2]:
from implicit.bpr import BayesianPersonalizedRanking

### Функция предварительной фильтрации

In [3]:
# Предполагаем, что на входе данные, как минимум,
# со столбцами user_id, item_id, quantity, sales_value, week_no, commodity_desc
def prefilter_items(data, prevalence_range = (0.05, 0.95)):
    # Уберем самые популярные товары (их и так купят)
    # Уберем также и непопулярные товары
    # Отсечем товары, которые вместе дают по 5% от общего числа попадания товаров в корзину покупателя
    # как со стороны популярных, так со стороны непопулярных товаров
    pop_thr, unpop_thr = prevalence_range
    item_cum_counts = data['item_id'].value_counts().cumsum()
    max_count = item_cum_counts.values[-1]
    top_popular_mask = item_cum_counts < max_count * pop_thr
    top_uppopular_mask = item_cum_counts > max_count * unpop_thr
    blocked_items = item_cum_counts[top_popular_mask | top_uppopular_mask].index
    
    # Уберем товары, которые не продавались за последние 12 месяцев
    recent_sale_items = data['item_id'][data['week_no'] > data['week_no'].max() - 53]
    old_sale_items = np.setdiff1d(data['item_id'], recent_sale_items)
    blocked_items = np.union1d(blocked_items, old_sale_items)
    
    # Уберем не интересные для рекоммендаций категории (department)
    # В данном датасете категория - commodity_desc, а не department
    # Мы не можем исключать категории на базе результатов продаж или других метрик
    # Префильтрация категорий - только на базе запроса бизнеса или legal
    # Отфильтруем несколько категорий для примера
    blocked_commodity = ['CIGARETTES', 'CIGARS', 'FAMILY PLANNING', 'LIQUOR', 'TOBACCO OTHER']
    
    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб.    
    # Уберем слишком дорогие товары
    # Желательно иметь данные о list price товара, попробуем косвенно оценить price по sales_value
    min_price, max_price = 1.0, 100.0
    bad_price_items = (
        data
        .assign(price = lambda x: np.where(x['quantity'] > 0, x['sales_value'] / x['quantity'], 0.0))
        .groupby('item_id')
        .agg(min_item_price=('price', 'min'), max_item_price=('price', 'max'))
        .query("min_item_price >= @max_price or max_item_price <= @min_price")
        .index
    )
    blocked_items = np.union1d(blocked_items, bad_price_items)
    
    # ...
    block_mask = (
        np.isin(data['item_id'], blocked_items) |
        np.isin(data['commodity_desc'], blocked_commodity)
    )
    result = data[~block_mask].copy()
    return result

### Данные и модель

Для тестирования работы других функций практической работы нужны подготовленные данные, матрица user/item и построенная модель

In [4]:
TRANSACTION_DATA = "../hw3/transaction_data_filtered.csv.zip"
PRODUCT_DATA = "../hw2/product.csv"

In [5]:
def read_transform_csv(path, column_map={}, index=None):
    column_names = pd.read_csv(path, nrows=0).columns
    _column_map = {col: col.lower() for col in column_names}
    _column_map.update(column_map)
    _data = pd.read_csv(path).rename(columns=_column_map)
    if index is not None:
        return _data.set_index(index)    
    return _data

In [6]:
%%time
# ДАННЫЕ
# product data
item_features = read_transform_csv(PRODUCT_DATA, {'PRODUCT_ID': 'item_id'}, index='item_id')

# transaction data
merged_data = pd.merge(pd.read_csv(TRANSACTION_DATA), item_features, on='item_id')

# prefiltered data
required_columns = ['user_id', 'item_id', 'quantity', 'sales_value', 'week_no', 'department', 'commodity_desc']
data = prefilter_items(merged_data.filter(required_columns))

# train-test split
test_size_weeks = 3
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# Actual test items
actual = data_test.groupby('user_id').agg(actual=('item_id', list))

# Матрица USER/ITEM
user_item_df = (
    data_train
    .groupby(['user_id', 'item_id'])
    .agg({'quantity': 'count'})
    .reset_index()
)

# Мапинг id <-> matrix index
user_id_unique = user_item_df['user_id'].unique()  # index -> user id
item_id_unique = user_item_df['item_id'].unique()  # index -> item id
from_uid = {uid: i for i, uid in enumerate(user_id_unique)}  # user id -> index
from_iid = {iid: i for i, iid in enumerate(item_id_unique)}  # item id -> index

user_item_matrix = bsr_matrix(
    (user_item_df['quantity'].astype(float),  # data
     (user_item_df['user_id'].map(from_uid),  # row
      user_item_df['item_id'].map(from_iid))),  # col
    shape=(len(from_uid), len(from_iid)))

sparse_user_item = user_item_matrix.tocsr()
sparse_item_user = user_item_matrix.T.tocsr()

Wall time: 5.15 s


In [7]:
merged_data.item_id.nunique(), data.item_id.nunique()

(92339, 25756)

In [8]:
%%time
# МОДЕЛЬ
model = BayesianPersonalizedRanking(factors=64, 
                                regularization=0.05,
                                learning_rate=0.01,
                                iterations=25, 
                                num_threads=4)

model.fit(sparse_item_user, show_progress=True)

  0%|          | 0/25 [00:00<?, ?it/s]

Wall time: 8.55 s


In [9]:
# Посмотрим для примера покупки и рекомендации для одного клиента
LOOKUP_UID = 1598
# Продукты, которые покупал этот клиент
actual_user_items = item_features.loc[actual.loc[LOOKUP_UID, 'actual']]
# Покажем только уникальные разделы, в которых совершались покупки
actual_user_items['department'].unique()

array(['PRODUCE', 'GROCERY', 'MEAT-PCKGD', 'DELI', 'SALAD BAR', 'DRUG GM',
       'MEAT', 'PASTRY'], dtype=object)

In [10]:
def get_similar_items_recommendation(user_items_interactions, model, all_item_ids, N=5):
    """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
    
    top_items = np.argsort(-user_items_interactions)[:N]
    similar_items = [model.similar_items(itm)[1][0] for itm in top_items]
    return all_item_ids[similar_items]

In [11]:
def get_user_item_matrix_row(user_idx, user_item_matrix):
    return (
        user_item_matrix
        .tocsr()
        .getrow(user_idx)
        .toarray()
        .flatten()
    )

In [12]:
%%time
LOOKUP_UID = 1598
user_items_interactions = get_user_item_matrix_row(from_uid[LOOKUP_UID], user_item_matrix)
similar_items_rec = get_similar_items_recommendation(user_items_interactions, model, item_id_unique, 5)
item_features.loc[similar_items_rec]

Wall time: 44 ms


,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
1076056,5,GROCERY,Private,COUPON/MISC ITEMS,MISC SALES TRANS,
1014948,69,PASTRY,Private,COOKIES,COOKIES: REGULAR,
913785,131,GROCERY,National,CONDIMENTS/SAUCES,CATSUP,24 OZ
944317,1071,GROCERY,National,FROZEN PIZZA,SANDWICHES&HANDHELDS,9 OZ
1004097,1107,MEAT-PCKGD,National,LUNCHMEAT,POULTRY,6 OZ


In [13]:
def get_similar_users_recommendation(user_idx, model, all_item_ids, N=5):
    """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    
    similar_users, similarity = zip(*model.similar_users(user_idx, N=N+1)[1:])
    user_weights = np.array(similarity) / np.sum(similarity)
    similar_recs = model.user_factors[similar_users, :] @ model.item_factors.T
    return all_item_ids[np.argsort(-user_weights @ similar_recs)[:N]]

In [14]:
%%time
similar_users_rec = get_similar_users_recommendation(from_uid[LOOKUP_UID], model, item_id_unique, 5)
item_features.loc[similar_users_rec]

Wall time: 9.99 ms


,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
1098066,69,GROCERY,Private,BAKED BREAD/BUNS/ROLLS,HOT DOG BUNS,11 OZ
1044078,2845,MEAT,National,BEEF,LEAN,
1004906,69,PRODUCE,Private,POTATOES,POTATOES RUSSET (BULK&BAG),5 LB
866211,2,PRODUCE,National,GRAPES,GRAPES WHITE,18 LB
1127831,5937,PRODUCE,National,BERRIES,STRAWBERRIES,16 OZ


На первый взгляд рекомендации не выбиваются и круга привычных покупок пользователя.